# Forecasting 4: Comparing Forecasting Techniques

The following notebook will compare the AutoARIMA, Prophet, and XGBoost models for predicting weather forecasts. The goal of this is to determine weighting for a mixed approach, where forecasts use weighted predictions of each model that are then averaged to produce a (hopefully) more robust measure.

Let's start by importing packages and getting a couple of test datasets, then developing each of the models.

## 1. Importing Packages and Data

### Defining Required Packages and Region Information

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from backend.regionweather import RegionWeather
from backend.autoarima import AutoARIMAForecast
from backend.prophet import ProphetForecast
from backend.xgb import XGBoostForecast

# Get the weather data for the region
region_data = {
    'US-FLA-FMPP': {'lat': 28.525581, 'lon': -81.536775, 'alt': 0},
    'US-FLA-FPC': {'lat': 28.996695, 'lon': -82.886613, 'alt': 0},
    'US-FLA-FPL': {'lat': 27.917488, 'lon': -81.450970, 'alt': 0},
    'US-FLA-GVL': {'lat': 29.619310, 'lon': -82.328732, 'alt': 0},
    'US-FLA-HST': {'lat': 25.456904, 'lon': -80.588092, 'alt': 0},
    'US-FLA-JEA': {'lat': 30.390902, 'lon': -83.679837, 'alt': 0},
    'US-FLA-SEC': {'lat': 28.805983, 'lon': -82.306291, 'alt': 0},
    'US-FLA-TAL': {'lat': 30.437174, 'lon': -84.248042, 'alt': 0},
    'US-FLA-TEC': {'lat': 27.959413, 'lon': -82.144821, 'alt': 0}
}

### Create RegionWeather Entities

In [2]:
region_name = list(region_data.keys())[0]
region_lat = region_data[region_name]['lat']
region_lon = region_data[region_name]['lon']
region_alt = region_data[region_name]['alt']

end = dt.datetime.now()
start = end - dt.timedelta(days=5)

# Get the weather data for the region
region_weather = RegionWeather(region_name, region_lat, region_lon, region_alt, start, end).to_dict()

fifteen = region_weather['df_15m']
hourly = region_weather['df_hourly']
daily = region_weather['df_daily']

Fetching Hourly Object...
Hourly Object Fetched!
Fetching Hourly Data from Object...
Hourly Object Fetched!
Hourly Data Cleaned!
Fetching Daily Data...
Daily Data Fetched!
Fetching Weekly Data...
Weekly Data Aggregated and Fetched!
Fetching Monthly Data...
Fetching Daily Data...
Monthly Data Fetched!
15 Minute Data Interpolated!


### Create Temperature Objects

Daily Data is still broken for some reason, I'll have to go back and troubleshoot that later...

In [3]:
temp_15m = fifteen['temp'].reset_index().dropna()
temp_15m.columns = ['ds', 'y']
temp_15m['unique_id'] = '1'

temp_hourly = hourly['temp'].reset_index().dropna()
temp_hourly.columns = ['ds', 'y']
temp_hourly['unique_id'] = '2'

In [4]:
temp_15m.head()

,ds,y,unique_id
0,2025-03-24 00:00:00,22.20,1
1,2025-03-24 00:15:00,21.85,1
2,2025-03-24 00:30:00,21.50,1
3,2025-03-24 00:45:00,21.15,1
4,2025-03-24 01:00:00,20.80,1


In [5]:
temp_hourly.head()

,ds,y,unique_id
0,2025-03-24 00:00:00,22.2,2
1,2025-03-24 01:00:00,20.8,2
2,2025-03-24 02:00:00,21.2,2
3,2025-03-24 03:00:00,19.6,2
4,2025-03-24 04:00:00,19.2,2


## Model 1: AutoARIMA

Now that the data is setup, let's get started forecasting with the AutoARIMA class.

In [7]:
aarima_15m = AutoARIMAForecast(region_name, temp_15m)
aarima_15m._fit()
aarima_15m.forecast(horizon=96)
aarima_15m.plot_forecast(horizon=96)

aarima_15m_res = aarima_15m.evaluate_model()

Validating Data
Initializing Model
Initializing StatsForecast
Fitting Model


Model Fitted


TypeError: StatsForecast.forecast() got an unexpected keyword argument 'horizon'

In [ ]:
aarima_hourly = AutoARIMAForecast(region_name, temp_hourly)
aarima_hourly.fit()
aarima_hourly.forecast(horizon=24)
aarima_hourly.plot_forecast(horizon=24)

aarima_hourly_res = aarima_hourly.evaluate_model()